In [1]:
%matplotlib inline

import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys

from PIL import Image
from utils.helpers import *

In [2]:
IMG_PATCH_SIZE = 16

In [3]:
# Extract patches from a given image
def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0, imgheight, h):
        for j in range(0, imgwidth, w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

In [4]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.25  # percentage of pixels > 1 required to assign a foreground label to a patch
    df = np.sum(v)
    if df > foreground_threshold:  # road
        return [0, 1]
    else:  # bgrd
        return [1, 0]

In [5]:
# Extract label images
def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images + 1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)


In [6]:
def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return np.asarray(data)

In [7]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/'

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, 2)
train_labels = extract_labels(train_labels_filename, 2)

Loading data/training/images/satImage_001.png
Loading data/training/images/satImage_002.png
Loading data/training/groundtruth/satImage_001.png
Loading data/training/groundtruth/satImage_002.png


In [8]:
train_data.shape

(1250, 16, 16, 3)

In [9]:
train_labels.shape

(1250, 2)

In [10]:
BATCH_SIZE = train_data.shape[0]

In [11]:
from src.unet_keras import *

/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/olivier/anaconda3/envs/ml/lib/python3.7/site-

In [12]:
unet_model(400, 3, 64, 3)

Instructions for updating:
Colocations handled automatically by placer.


In [31]:
# Loaded a set of images
root_dir = "data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(5, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

Loading 5 images
satImage_052.png
Loading 5 images
satImage_052.png


In [32]:
np.asarray(imgs).shape

(5, 400, 400, 3)

In [35]:
gt_imgs_test = np.asarray(gt_imgs)

In [36]:
gt_imgs_test.shape

(5, 400, 400)

In [39]:
test = np.expand_dims(gt_imgs_test, axis=3)

In [40]:
test.shape

(5, 400, 400, 1)

In [41]:
x_train = np.asarray(imgs)
y_train = np.expand_dims(np.asarray(gt_imgs), axis=3)

In [42]:
x_train.shape

(5, 400, 400, 3)

In [43]:
y_train.shape

(5, 400, 400, 1)

In [23]:
IMG_SIZE = 400
NUM_CHANNELS = 3
NUM_FILTER = 16
FILTER_SIZE = 3

BATCH_SIZE = 2
NUM_EPOCHS = 5

In [24]:
model = unet_model(IMG_SIZE, NUM_CHANNELS, NUM_FILTER, FILTER_SIZE)

In [45]:
model = train_model(model, x_train, y_train, x_train[:2], y_train[:2], BATCH_SIZE, NUM_EPOCHS)

Train on 5 samples, validate on 2 samples
Epoch 1/5
5/5 [==============================] - 9s 2s/sample - loss: 1.0997 - acc: 0.6051 - val_loss: 1.1892 - val_acc: 0.7350
Epoch 2/5
5/5 [==============================] - 7s 1s/sample - loss: 1.0196 - acc: 0.6145 - val_loss: 1.1151 - val_acc: 0.7501
Epoch 3/5
5/5 [==============================] - 7s 1s/sample - loss: 0.9647 - acc: 0.6260 - val_loss: 1.1136 - val_acc: 0.7577
Epoch 4/5
5/5 [==============================] - 7s 1s/sample - loss: 0.9076 - acc: 0.6581 - val_loss: 1.1970 - val_acc: 0.7737
Epoch 5/5
5/5 [==============================] - 7s 1s/sample - loss: 0.8974 - acc: 0.6912 - val_loss: 1.2588 - val_acc: 0.7686

history dict: {'loss': [1.0996918678283691, 1.0196349620819092, 0.9647100448608399, 0.9075574517250061, 0.8974032521247863], 'acc': [0.6051412, 0.614485, 0.6259775, 0.658075, 0.69123], 'val_loss': [1.1892069578170776, 1.1150537729263306, 1.113621711730957, 1.1970491409301758, 1.2587614059448242], 'val_acc': [0.73503